In [1]:
import pandas as pd
from pickle import load
pd.set_option("max_rows", None)

In [2]:
# Templates for finding stuff
# df_race.where(df['County Name'] is in df_race['County'].values).notna()
# df['County Name'].isin(df_race['County']).value_counts()
# (df.loc[df['County Name'].str.contains('Oglala', case=False)])
# df_race.loc[~df_race['County'].isin(df['County Name'])]

In [3]:
df_race = load(open('df_race_alaska_fixed.pkl', 'rb'))

In [4]:
df = pd.read_csv('2016 Presidential Election Analysis.csv')

In [6]:
us_state_abbrev = {
    'AL': 'Alabama',
    'AK': 'Alaska',
    'AZ': 'Arizona',
    'AR': 'Arkansas',
    'CA': 'California',
    'CO': 'Colorado',
    'CT': 'Connecticut',
    'DE': 'Delaware',
    'FL': 'Florida',
    'GA': 'Georgia',
    'HI': 'Hawaii',
    'ID': 'Idaho',
    'IL': 'Illinois',
    'IN': 'Indiana',
    'IA': 'Iowa',
    'KS': 'Kansas',
    'KY': 'Kentucky',
    'LA': 'Louisiana',
    'ME': 'Maine',
    'MD': 'Maryland',
    'MA': 'Massachusetts',
    'MI': 'Michigan',
    'MN': 'Minnesota',
    'MS': 'Mississippi',
    'MO': 'Missouri',
    'MT': 'Montana',
    'NE': 'Nebraska',
    'NV': 'Nevada',
    'NH': 'New Hampshire',
    'NJ': 'New Jersey',
    'NM': 'New Mexico',
    'NY': 'New York',
    'NC': 'North Carolina',
    'ND': 'North Dakota',
    'OH': 'Ohio',
    'OK': 'Oklahoma',
    'OR': 'Oregon',
    'PA': 'Pennsylvania',
    'RI': 'Rhode Island',
    'SC': 'South Carolina',
    'SD': 'South Dakota',
    'TN': 'Tennessee',
    'TX': 'Texas',
    'UT': 'Utah',
    'VT': 'Vermont',
    'VA': 'Virginia',
    'WA': 'Washington',
    'WV': 'West Virginia',
    'WI': 'Wisconsin',
    'WY': 'Wyoming',
}

In [7]:
df['County Name'] = df['County Name'].str.rstrip('County')

In [8]:
df = df.replace('Oglala', 'Oglala Lakota', regex = True)

In [9]:
df = df.replace('La Salle Parish', 'LaSalle Parish', regex = True)

In [10]:
len(df_race.State.value_counts())

51

In [11]:
len(df['State Code'].value_counts())

51

In [12]:
missing = df.loc[~df['County Name'].isin(df_race['County'])]
missing

,State Code,County Name,County Population,Clinton or Trump State,Clinton,Trump,Total,% Clinton,% Trump,Vote Difference C-T,...,County FIPS Code,CBSA Code,CSA Code,CBSA Title,CBSA Population,Metropolitan/Micropolitan Statistical Area,CSA Title,Metropolitan Division Title,Metropolitan Division Code,Central/Outlying County


In [13]:
df_race.loc[~df_race['County'].isin(df['County Name'])]

1,id,Total,TotalPopulation of one race,TotalPopulation of one raceWhite alone,TotalPopulation of one raceBlack or African American alone,TotalPopulation of one raceAmerican Indian and Alaska Native alone,TotalPopulation of one raceAsian alone,TotalPopulation of one raceNative Hawaiian and Other Pacific Islander alone,TotalPopulation of one raceSome Other Race alone,TotalPopulation of two or more races,County,State


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3113 entries, 0 to 3112
Data columns (total 39 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   State Code                                  3113 non-null   object 
 1   County Name                                 3113 non-null   object 
 2    County Population                          3112 non-null   object 
 3   Clinton or Trump State                      3113 non-null   object 
 4    Clinton                                    3113 non-null   object 
 5    Trump                                      3113 non-null   object 
 6    Total                                      3113 non-null   object 
 7   % Clinton                                   3113 non-null   object 
 8   % Trump                                     3113 non-null   object 
 9    Vote Difference C-T                        3113 non-null   object 
 10   Vote Differ